# Meta Ailesi Modelleriyle Çalışmak

## Giriş

Bu derste şunları öğreneceksiniz:

- Meta ailesinin iki ana modelini keşfetmek - Llama 3.1 ve Llama 3.2
- Her modelin kullanım alanlarını ve senaryolarını anlamak
- Her modelin kendine özgü özelliklerini gösteren bir kod örneği

## Meta Model Ailesi

Bu derste, Meta ailesinden veya "Llama Sürüsü"nden 2 modeli inceleyeceğiz - Llama 3.1 ve Llama 3.2

Bu modeller farklı varyantlarda gelir ve Github Model pazarında bulunabilir. Github Modelleri kullanarak [AI modelleriyle prototipleme](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) hakkında daha fazla bilgiye buradan ulaşabilirsiniz.

Model Varyantları:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Not: Llama 3 de Github Modellerde mevcut, ancak bu derste ele alınmayacak*


## Llama 3.1

405 Milyar Parametre ile Llama 3.1, açık kaynak LLM kategorisine giriyor.

Bu model, önceki Llama 3 sürümüne göre şu geliştirmeleri sunuyor:

- Daha büyük bağlam penceresi - 128k token, önceki 8k tokene kıyasla
- Daha yüksek Maksimum Çıktı Tokenı - 4096, önceki 2048'e kıyasla
- Daha iyi Çok Dilli Destek - eğitim tokenlarının artması sayesinde

Bunlar, Llama 3.1’in GenAI uygulamaları geliştirirken daha karmaşık kullanım senaryolarını yönetmesini sağlıyor. Bunlar arasında:
- Yerel Fonksiyon Çağırma - LLM iş akışının dışında harici araç ve fonksiyonları çağırabilme yeteneği
- Daha İyi RAG Performansı - daha yüksek bağlam penceresi sayesinde
- Sentetik Veri Üretimi - ince ayar gibi görevler için etkili veri oluşturabilme yeteneği


### Yerel Fonksiyon Çağrısı

Llama 3.1, fonksiyon veya araç çağrıları yapmada daha etkili olacak şekilde ince ayarlandı. Ayrıca, modelin kullanıcıdan gelen isteğe göre kullanılması gerektiğini anlayabileceği iki yerleşik aracı bulunuyor. Bu araçlar:

- **Brave Search** - Web araması yaparak hava durumu gibi güncel bilgilere ulaşmak için kullanılabilir
- **Wolfram Alpha** - Daha karmaşık matematiksel hesaplamalar için kullanılabilir, böylece kendi fonksiyonlarınızı yazmanıza gerek kalmaz.

Ayrıca, LLM'nin çağırabileceği kendi özel araçlarınızı da oluşturabilirsiniz.

Aşağıdaki kod örneğinde:

- Kullanılabilir araçları (brave_search, wolfram_alpha) sistem isteminde tanımlıyoruz.
- Belirli bir şehirdeki hava durumu hakkında soru soran bir kullanıcı istemi gönderiyoruz.
- LLM, Brave Search aracına bir araç çağrısı ile yanıt verecek ve bu şekilde görünecek: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Not: Bu örnek yalnızca araç çağrısı yapar, sonuçları almak isterseniz Brave API sayfasında ücretsiz bir hesap oluşturmanız ve fonksiyonu kendiniz tanımlamanız gerekir.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Llama 3.1 bir LLM olmasına rağmen, sahip olduğu sınırlamalardan biri çok modluluk özelliğinin olmamasıdır. Yani, farklı türde girdileri (örneğin görselleri) istem olarak kullanıp yanıt verebilme yeteneği eksiktir. Bu yetenek, Llama 3.2'nin en önemli özelliklerinden biridir. Bu özellikler ayrıca şunları da içerir:

- Çok modluluk - hem metin hem de görsel istemleri değerlendirme yeteneğine sahiptir
- Küçük ve Orta boyutlu varyasyonlar (11B ve 90B) - bu, esnek dağıtım seçenekleri sunar,
- Sadece metin varyasyonları (1B ve 3B) - bu, modelin uç / mobil cihazlarda dağıtılmasına ve düşük gecikme sağlamasına olanak tanır

Çok modlu destek, açık kaynak modeller dünyasında büyük bir adımı temsil ediyor. Aşağıdaki kod örneği, hem bir görsel hem de metin istemi alarak Llama 3.2 90B'den görselin analizini elde ediyor.

### Llama 3.2 ile Çok Modlu Destek


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Öğrenme burada bitmiyor, yolculuğa devam et

Bu dersi tamamladıktan sonra, Generative AI bilginizi geliştirmeye devam etmek için [Generative AI Öğrenme koleksiyonumuza](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) göz atmayı unutmayın!



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerde hata veya yanlışlıklar bulunabileceğini lütfen unutmayın. Belgenin orijinal diliyle hazırlanmış hali esas alınmalıdır. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından doğabilecek yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
